# Mencari Akurasi Terbaik Antara Data Yang Belum di Normalisasi dengan Data Yang Sudah di Normalisasi

In [28]:
# mengimpor paket pandas kemudian diberi nama alias pd
import pandas as pd
# mengimpor paket numpy kemudian diberi nama alias np
import numpy as np
# untuk membaca file arff
from scipy.io import arff


# Modelling
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# untuk menghilangkan warning
import warnings
warnings.filterwarnings('ignore')

## Import Dataset

In [29]:
# membaca dataset arff dari url
data = arff.loadarff('/content/drive/MyDrive/datamining/tugas/messidor_features.arff')
df = pd.DataFrame(data[0])

In [30]:
col_names = []
for i in range(20):
    if i == 0:
        col_names.append('quality')
    if i == 1:
        col_names.append('prescreen')
    if i >= 2 and i <= 7:
        col_names.append('ma' + str(i))
    if i >= 8 and i <= 15:
        col_names.append('exudate' + str(i))
    if i == 16:
        col_names.append('euDist')
    if i == 17:
        col_names.append('diameter')
    if i == 18:
        col_names.append('amfm_class')
    if i == 19:
        col_names.append('label')

In [31]:
df.columns = [col_names]
df

,quality,prescreen,ma2,ma3,ma4,ma5,ma6,ma7,exudate8,exudate9,exudate10,exudate11,exudate12,exudate13,exudate14,exudate15,euDist,diameter,amfm_class,label
0,1.0,1.0,22.0,22.0,22.0,19.0,18.0,14.0,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1.0,b'0'
1,1.0,1.0,24.0,24.0,22.0,18.0,16.0,13.0,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0.0,b'0'
2,1.0,1.0,62.0,60.0,59.0,54.0,47.0,33.0,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0.0,b'1'
3,1.0,1.0,55.0,53.0,53.0,50.0,43.0,31.0,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0.0,b'0'
4,1.0,1.0,44.0,44.0,44.0,41.0,39.0,27.0,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0.0,b'1'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,1.0,1.0,34.0,34.0,34.0,33.0,31.0,24.0,6.071765,0.937472,0.031145,0.003115,0.000000,0.000000,0.000000,0.000000,0.537470,0.116795,0.0,b'0'
1147,1.0,1.0,49.0,49.0,49.0,49.0,45.0,37.0,63.197145,27.377668,8.067688,0.979548,0.001552,0.000000,0.000000,0.000000,0.516733,0.124190,0.0,b'0'
1148,1.0,0.0,49.0,48.0,48.0,45.0,43.0,33.0,30.461898,13.966980,1.763305,0.137858,0.011221,0.000000,0.000000,0.000000,0.560632,0.129843,0.0,b'0'
1149,1.0,1.0,39.0,36.0,29.0,23.0,13.0,7.0,40.525739,12.604947,4.740919,1.077570,0.563518,0.326860,0.239568,0.174584,0.485972,0.106690,1.0,b'1'


In [32]:
X = df.drop(columns="label")
X

,quality,prescreen,ma2,ma3,ma4,ma5,ma6,ma7,exudate8,exudate9,exudate10,exudate11,exudate12,exudate13,exudate14,exudate15,euDist,diameter,amfm_class
0,1.0,1.0,22.0,22.0,22.0,19.0,18.0,14.0,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1.0
1,1.0,1.0,24.0,24.0,22.0,18.0,16.0,13.0,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0.0
2,1.0,1.0,62.0,60.0,59.0,54.0,47.0,33.0,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0.0
3,1.0,1.0,55.0,53.0,53.0,50.0,43.0,31.0,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0.0
4,1.0,1.0,44.0,44.0,44.0,41.0,39.0,27.0,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,1.0,1.0,34.0,34.0,34.0,33.0,31.0,24.0,6.071765,0.937472,0.031145,0.003115,0.000000,0.000000,0.000000,0.000000,0.537470,0.116795,0.0
1147,1.0,1.0,49.0,49.0,49.0,49.0,45.0,37.0,63.197145,27.377668,8.067688,0.979548,0.001552,0.000000,0.000000,0.000000,0.516733,0.124190,0.0
1148,1.0,0.0,49.0,48.0,48.0,45.0,43.0,33.0,30.461898,13.966980,1.763305,0.137858,0.011221,0.000000,0.000000,0.000000,0.560632,0.129843,0.0
1149,1.0,1.0,39.0,36.0,29.0,23.0,13.0,7.0,40.525739,12.604947,4.740919,1.077570,0.563518,0.326860,0.239568,0.174584,0.485972,0.106690,1.0


In [33]:
y = df['label'].values
y[0:5]

array([[b'0'],
       [b'0'],
       [b'1'],
       [b'0'],
       [b'1']], dtype=object)

## Akurasi Data Yang Belum Di Normalisasi (Naive Bayes)

### Encoder Label

In [34]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
yBaru=le.transform(y)

### Split Dataset

In [35]:
# membagi data menjadi set train dan test (80:20)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X , yBaru, test_size=0.2, shuffle=False)

### Akurasi

In [36]:
# classifier
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)

post_nb = gaussian.predict_proba(X_test)
probas_nb = post_nb[:,1]
probas_nb = np.round(probas_nb)

pred_nb = probas_nb
accuracy_nb = accuracy_score(y_test, pred_nb)
print('Akurasi Data yang belum di normalisasi menggunakan Naive Bayes adalah = %.3f' %accuracy_nb)

Akurasi Data yang belum di normalisasi menggunakan Naive Bayes adalah = 0.645


## Akurasi Data Yang Sudah Di Normalisasi (Naive Bayes)

$$ x'=\frac{x-x_{min}}{x_{max}-x_{min}}$$

### Save Model

In [37]:
import joblib
scaler = MinMaxScaler()
#scaler.fit(features)
#scaler.transform(features)
scaled = scaler.fit_transform(X)
features_names = X.columns.copy()
#features_names.remove('label')
scaled_features = pd.DataFrame(scaled, columns=features_names)
scaled_features.head(10)
scaler_filename = "scaled.save"
joblib.dump(scaler, scaler_filename) 
scaler = joblib.load(scaler_filename)

### Encoder Label

In [38]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
yBaru=le.transform(y)

### Split Dataset

In [39]:
# membagi data menjadi set train dan test (80:20)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled , yBaru, test_size=0.2, shuffle=False)

### Akurasi

In [40]:
# classifier
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)

post_nb = gaussian.predict_proba(X_test)
probas_nb = post_nb[:,1]
probas_nb = np.round(probas_nb)

pred_nb = probas_nb
accuracy_nb = accuracy_score(y_test, pred_nb)
print('Akurasi Data yang sudah di normalisasi menggunakan Naive Bayes adalah = %.3f' %accuracy_nb)

Akurasi Data yang sudah di normalisasi menggunakan Naive Bayes adalah = 0.636


## Akurasi Data Yang Belum Di Normalisasi (KNN)

### Encoder Label

In [41]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y=le.transform(y)

### Split Dataset

In [42]:
# membagi data menjadi set train dan test (80:20)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X , yBaru, test_size=0.2, shuffle=False)

In [43]:
Jumlah_K = 10

### Akurasi

In [44]:
# classifier
knn = KNeighborsClassifier(Jumlah_K)
knn.fit(X_train, y_train)

post_knn = knn.predict_proba(X_test)
probas_knn = post_knn[:,1]
probas_knn = np.round(probas_knn)

pred_knn = probas_knn
accuracy_knn = accuracy_score(y_test, pred_knn)
print('Akurasi Data yang belum di normalisasi menggunakan KNN adalah =  %.3f' %accuracy_knn)

Akurasi Data yang belum di normalisasi menggunakan KNN adalah =  0.662


## Akurasi Data Yang Sudah Di Normalisasi (KNN)

$$ x'=\frac{x-x_{min}}{x_{max}-x_{min}}$$

### Save Model

In [45]:
import joblib
scaler = MinMaxScaler()
#scaler.fit(features)
#scaler.transform(features)
scaled = scaler.fit_transform(X)
features_names = X.columns.copy()
#features_names.remove('label')
scaled_features = pd.DataFrame(scaled, columns=features_names)
scaled_features.head(10)
scaler_filename = "scaled.save"
joblib.dump(scaler, scaler_filename) 
scaler = joblib.load(scaler_filename)

### Encoder Label

In [46]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
yBaru=le.transform(y)

### Split Dataset

In [47]:
# membagi data menjadi set train dan test (80:20)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled , yBaru, test_size=0.2, shuffle=False)

In [48]:
Jumlah_K = 10

### Akurasi

In [49]:
# classifier
knn = KNeighborsClassifier(Jumlah_K)
knn.fit(X_train, y_train)

post_knn = knn.predict_proba(X_test)
probas_knn = post_knn[:,1]
probas_knn = np.round(probas_knn)

pred_knn = probas_knn
accuracy_knn = accuracy_score(y_test, pred_knn)
print('Akurasi Data yang sudah di normalisasi menggunakan KNN adalah =  %.3f' %accuracy_knn)

Akurasi Data yang sudah di normalisasi menggunakan KNN adalah =  0.684
